In [1]:
import numpy as np
import os
import pandas as pd
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

In [2]:
maindir = os.getcwd()

In [3]:
os.chdir("/../../datasets/aclImdb/")

In [4]:
voc_f = open("imdb.vocab",'r')
vocab = voc_f.read().split("\n")

In [5]:
Exp_f = open("imdbEr.txt",'r')
EV = Exp_f.read().split("\n")[:-1]

In [6]:
vocabInfo = pd.DataFrame({"vocab":vocab,"expected":EV})
vocabInfo

,vocab,expected
0,the,0.0490972013402
1,and,0.201363575849
2,a,0.0333946807184
3,of,0.099837669572
4,to,-0.0790210365788
...,...,...
89522,copywrite,0
89523,artbox,0
89524,kinky-sex,0
89525,urrrghhh,0


In [7]:
vocabInfo = vocabInfo[~ vocabInfo["vocab"].isin(en_stops)]
vocabInfo["expected"] = abs(vocabInfo["expected"].astype(float))
vocabInfo

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,vocab,expected
15,movie,0.246354
17,film,0.147886
26,one,0.087114
27,!,0.134839
36,like,0.280548
...,...,...
89522,copywrite,0.000000
89523,artbox,0.000000
89524,kinky-sex,0.000000
89525,urrrghhh,0.000000


In [8]:
vocabInfo.sort_values(by="expected",ascending=False)[:round(len(vocabInfo)*0.05)]

,vocab,expected
54867,pelswick,4.500000
33943,magorian,4.500000
67306,subspace,4.500000
35092,jgar,4.500000
58224,buh,4.500000
...,...,...
81702,danira,2.267657
61418,ecclesiastical,2.267421
40382,grey-haired,2.267421
44828,irland,2.267421


In [9]:
sub = (vocabInfo[vocabInfo["expected"] <= 2].sort_values(by="expected"))
reducedVocab = sub.sample(n=round(len(sub)*0.1),random_state=1)
reducedVocab

,vocab,expected
55266,telethons,0.000000
23061,bea,0.111697
23619,loulou,1.944736
59561,code-approved,0.000000
61309,yonfan,0.000000
...,...,...
50128,visability,0.000000
33859,big-boned,0.000000
25123,darshan,1.234801
24780,tanker,0.355504


In [10]:
newVocab = pd.concat([vocabInfo[(vocabInfo.vocab == "good") | (vocabInfo.vocab=="bad")],reducedVocab])

# Train

In [11]:
os.chdir("train/")

In [12]:
f = open("labeledBow.feat",'r')
reviews = f.read().split("\n")
ratings = []
BOW = []
for i in reviews:
    elements = i.split(" ")
    ratings.append((elements[0]))
    BOW.append(elements[1:])
f.close()

In [13]:
BOW = [[j.split(":") for j in i] for i in BOW]

In [14]:
# Removing words
l=list(newVocab["vocab"])
for i in range(len(BOW)): #rev
    keep = {}
    for e in BOW[i]: #word
        if vocab[int(e[0])] in l:
            e[0] = vocab[int(e[0])]
            e[1] = int(e[1])
            keep[e[0]] = e[1]
    BOW[i] = keep
    

In [15]:
BOW = pd.Series(BOW)

In [16]:
BOW

0        {'life': 1, 'comedy': 1, 'whole': 1, 'reality'...
1        {'good': 3, 'also': 2, 'comedy': 1, 'stories':...
2                           {'cross': 1, 'correctness': 1}
3        {'something': 1, 'experiment': 1, 'participant...
4        {'also': 1, 'love': 3, 'something': 1, 'must':...
                               ...                        
24996    {'bad': 1, 'can't': 2, 'guy': 1, 'everything':...
24997    {'good': 2, 'also': 1, 'must': 1, 'run': 1, 'w...
24998    {'good': 3, 'whole': 1, 'female': 2, 'dimensio...
24999      {'good': 1, 'life': 1, 'face': 1, 'causing': 1}
25000                                                   {}
Length: 25001, dtype: object

In [17]:
tf = pd.DataFrame(list(BOW[:-1]))

In [18]:
tf = tf.fillna(0)
df = (tf > 0).sum(axis=0)
idf = np.log(len(tf) / df)
tf_idf_train = tf * idf
#np.save("idf.npy",idf)
# AFTER CAN JUMP TO TEST

In [19]:
ratings = np.array([int(i) for i in ratings[:-1]])
ratings = pd.Series((2*(ratings > 5).astype(int))-1)
tf_idf_train["Label"] =  ratings

In [20]:
tf_idf_train.head()

,life,comedy,whole,reality,cartoon,good,also,stories,behaved,cross,...,amazons-dr,now-over,creature-basically,mud-wrestling,multi-sexual,barabarian,gamorrean,d-grade,highly-entertaining,Label
0,1.734095,2.38033,2.250752,3.455865,4.257334,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000000,2.38033,0.000000,0.000000,0.000000,2.900542,2.707422,3.329807,7.082109,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.675593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [21]:
os.chdir(maindir)
os.chdir("..")
tf_idf_train.to_csv("tf_idf_train2.csv")

In [22]:
tf_idf_train

,life,comedy,whole,reality,cartoon,good,also,stories,behaved,cross,...,amazons-dr,now-over,creature-basically,mud-wrestling,multi-sexual,barabarian,gamorrean,d-grade,highly-entertaining,Label
0,1.734095,2.38033,2.250752,3.455865,4.257334,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
1,0.000000,2.38033,0.000000,0.000000,0.000000,2.900542,2.707422,3.329807,7.082109,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
2,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.675593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.000000,0.00000,0.000000,0.000000,0.000000,0.966847,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.126631,-1
24996,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1
24997,0.000000,0.00000,0.000000,0.000000,0.000000,1.933695,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1
24998,0.000000,0.00000,2.250752,0.000000,0.000000,2.900542,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1


# Test

In [23]:
os.chdir("/../../datasets/aclImdb/test")

In [24]:
f = open("labeledBow.feat",'r')
reviews = f.read().split("\n")
ratings = []
BOW = []
for i in reviews:
    elements = i.split(" ")
    ratings.append((elements[0]))
    BOW.append(elements[1:])
f.close()

In [25]:
BOW = [[j.split(":") for j in i] for i in BOW]

In [ ]:
# Removing words
l=list(newVocab["vocab"])
for i in range(len(BOW)): #rev
    keep = {}
    for e in BOW[i]: #word
        if vocab[int(e[0])] in l:
            e[0] = vocab[int(e[0])]
            e[1] = int(e[1])
            keep[e[0]] = e[1]
    BOW[i] = keep
    

In [ ]:
BOW = pd.Series(BOW)

In [ ]:
BOW

In [ ]:
tf = pd.DataFrame(list(BOW[:-1]))

In [ ]:
tf = tf.fillna(0)
df = (tf > 0).sum(axis=0)
idf = np.log(len(tf) / df)
tf_idf_test = tf * idf
#np.save("idf.npy",idf)
# AFTER CAN JUMP TO TEST

In [ ]:
ratings = np.array([int(i) for i in ratings[:-1]])
ratings = pd.Series((2*(ratings > 5).astype(int))-1)
tf_idf_test["Label"] =  ratings

In [ ]:
tf_idf_test.head()

In [ ]:
os.chdir(maindir)
os.chdir("..")
tf_idf_test.to_csv("tf_idf_test2.csv")

In [ ]:
tf_idf_test